In [29]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import pickle
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import pandas as pd
import numpy as np
import os

In [2]:
path = 'dataset/'

In [3]:
dataframe = pd.read_csv(path+'train.csv') 

In [4]:
dataframe.sample()

,filename,label
30177,30177.png,6


In [5]:
images = os.listdir(path+'train/')

In [6]:
len(images)

49000

In [7]:
class data_Label_Combiner:

    def Data_combiner(self,images, labels, path):
        x_train=[]
        y_train=[]
        for each in images:
            x_train.append(np.asarray(Image.open(path+each).resize((32,32)), dtype='int32'))
#             y_train.append(np.asarray(binarizer.transform(labels[labels['ID']==each]['Class']),dtype='float32'))
            y_train.append(np.asarray(labels[labels['filename']==each]['label']))
        return x_train, y_train


In [8]:
dlc= data_Label_Combiner()
x_train,y_train = dlc.Data_combiner(images,dataframe,path+'train/')

In [9]:
y_train[:5]

[array([5]), array([7]), array([6]), array([9]), array([2])]

In [10]:
a = np_utils.to_categorical(y_train[:5])

In [22]:
a.shape

(5, 10)

In [25]:
b = np.array(y_train)

In [26]:
b.shape

(49000, 1)

In [21]:
(np_utils.to_categorical(b[:6])).shape

(6, 10)

In [27]:
y_train = np.array(y_train)
x_train = np.array(x_train)
print(x_train.shape)
# x_train = x_train.reshape(-1,32,32,3)

(49000, 32, 32, 4)


In [30]:
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([5, 7, 6, 9, 2])

In [33]:
y_train = np_utils.to_categorical(y_train)

In [34]:
y_train.shape

(49000, 10)

In [35]:
x_train, x_cross_val, y_train, y_cross_val = train_test_split(x_train,y_train, test_size = 0.1)

In [12]:
# x_train.shape

In [36]:
# train_generator = ImageDataGenerator(rotation_range=0.2,zoom_range=0.3, shear_range=0.3, vertical_flip=True, rescale=1.0/255)

train_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    zoom_range=0.3,
    shear_range=0.3,
    rescale=1.0/255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

val_generator = ImageDataGenerator(rescale=1.0/255)

In [37]:
train_generator.fit(x_train)
val_generator.fit(x_cross_val)

In [38]:
classifier = Sequential()

In [39]:
classifier.add(Convolution2D(32,3,3, input_shape = (32,32,4),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 4..., activation="relu")`
  """Entry point for launching an IPython kernel.


Instructions for updating:
Colocations handled automatically by placer.


In [40]:
classifier.add(Flatten())

In [41]:
classifier.add(Dense(output_dim=120, activation='relu'))
classifier.add(Dense(output_dim =10 ,activation='softmax'))

/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120)`
  """Entry point for launching an IPython kernel.
/home/admin1/anaconda3/envs/prayas_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  


In [42]:
classifier.compile(optimizer='adam',loss = 'categorical_crossentropy' ,metrics=['accuracy'])

In [ ]:
# fits the model on batches with real-time data augmentation:
classifier.fit_generator(train_generator.flow(x_train,y_train),
                    steps_per_epoch=len(x_t),
                         epochs=10, validation_data=val_generator.flow(x_cross_val,y_cross_val), validation_steps=len(x_cross_val)
                    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
18375/44100 [===========>..................] - ETA: 18:29 - loss: 2.3014 - acc: 0.1128

In [ ]:
img = Image.fromarray(x_cross_val[1], 'RGB').resize((64,64))
img.show()

In [34]:
y_train.shape

(39690, 10, 2)

In [ ]:
with open('pickle/model.pkl','wb') as f:
    pickle.dump(val_generator,f)
    pickle.dump(clasifier,f)